In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, average_precision_score

In [2]:
df = pd.read_csv("combined_features.csv")

In [3]:
df = df.drop(["Unnamed: 0"],axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,Corr2,Diss_sim2,Homogen2,Contrast2,Energy3,Corr3,Diss_sim3,Contrast3,Energy4,Target
0,-0.000977,0.180385,-0.044725,0.150532,-0.217192,0.663009,0.318875,0.223675,0.251746,-0.094831,...,0.957856,5.260471,0.177639,45.961066,0.018837,0.897679,8.187458,111.380737,0.110231,0
1,0.195504,0.276669,-0.061317,0.407598,0.260970,0.825568,0.391509,0.583588,0.186765,0.156528,...,0.963605,6.100514,0.159296,64.673783,0.018169,0.922644,8.952959,137.322189,0.103154,0
2,0.111006,0.349559,-0.094262,0.216156,0.106058,1.137171,0.201154,0.361343,0.286866,0.052426,...,0.925942,6.688320,0.146016,75.827759,0.017696,0.844452,9.780711,158.963582,0.111965,0
3,0.100495,0.275781,-0.116318,0.311098,0.073257,0.908716,0.147829,0.179936,0.280274,0.042768,...,0.909106,6.583259,0.145556,72.746386,0.019040,0.800765,9.790649,159.140659,0.119870,0
4,-0.049671,0.119182,0.046339,0.119769,0.222893,0.303141,0.084352,0.012266,0.312024,0.056778,...,0.989274,1.503921,0.463249,3.939689,0.046894,0.973307,2.391478,9.771223,0.147577,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14160,0.092375,0.433341,0.148408,-0.140822,0.288990,0.349013,0.209123,-0.004986,0.133768,0.111640,...,0.999134,0.163963,0.918022,0.164001,0.128373,0.998540,0.270011,0.276910,0.167666,5
14161,-0.019356,0.083175,0.145959,-0.164610,0.223124,0.040336,-0.126574,-0.079431,0.380889,-0.066216,...,0.999425,0.041308,0.979346,0.041308,0.212875,0.999053,0.068443,0.068443,0.227692,5
14162,-0.124784,0.211899,-0.037131,-0.193507,0.178431,0.097447,0.016363,-0.115176,0.359523,-0.074620,...,0.998162,0.377889,0.814976,0.417155,0.097201,0.996178,0.623830,0.867390,0.166726,5
14163,0.137629,-0.007763,0.133060,0.196928,0.495559,0.170997,-0.038201,0.133344,0.366415,0.101528,...,0.998675,0.262682,0.869234,0.268435,0.134125,0.997359,0.436981,0.536186,0.183026,5


In [4]:
df["Target"].value_counts()

Target
1    4644
2    3288
5    1888
0    1885
4    1783
3     677
Name: count, dtype: int64

In [5]:
df["Target"].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [6]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['Target'])
y = df['Target']
# X = your feature matrix (excluding target column)
# Step 1: Scale the data (very important for PCA!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 2: Apply PCA
pca = PCA(n_components=100)  # Retain 95% of variance (OR set a fixed number like 50)
X_pca = pca.fit_transform(X_scaled)

print("Original shape:", X.shape)
print("Reduced shape:", X_pca.shape)

Original shape: (14165, 271)
Reduced shape: (14165, 100)


In [7]:


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Create and train the SVM model
model = SVC(kernel='rbf', C=1.0, gamma='scale',probability=True,class_weight='balanced')  # You can change kernel to 'linear', 'poly', etc.
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_train)

In [8]:
y_probs = model.predict_proba(X_train)

# 2. Convert y_test to binary format for average_precision_score
y_train_bin = label_binarize(y_train, classes=sorted(list(set(y))))

# 3. Compute mean Average Precision
mAP = average_precision_score(y_train_bin, y_probs, average="macro")
print("\n✅ mean Average Precision (mAP):", mAP)


✅ mean Average Precision (mAP): 0.9592258049056822


In [9]:
y_pred

array([2, 5, 1, ..., 1, 0, 2], dtype=int64)

In [10]:
print("Accuracy:", accuracy_score(y_train, y_pred))
print("\nClassification Report:\n", classification_report(y_train, y_pred))

Accuracy: 0.9231380162372044

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98      1492
           1       0.93      0.89      0.91      3696
           2       0.90      0.95      0.92      2617
           3       0.83      0.96      0.89       551
           4       0.97      0.99      0.98      1430
           5       0.88      0.85      0.87      1546

    accuracy                           0.92     11332
   macro avg       0.92      0.93      0.92     11332
weighted avg       0.92      0.92      0.92     11332



In [11]:
confusion_matrix(y_train, y_pred)

array([[1447,    0,   25,    1,   19,    0],
       [   5, 3281,  232,   10,    0,  168],
       [  10,    7, 2477,   92,   25,    6],
       [   1,    5,   13,  529,    0,    3],
       [   0,    4,   11,    4, 1410,    1],
       [   0,  221,    4,    3,    1, 1317]], dtype=int64)

In [12]:
X_pca


array([[ 7.71121631e+00,  5.20186080e+00, -1.21709172e+01, ...,
        -2.94505627e-02,  3.45144035e-01,  1.05450633e-01],
       [ 1.35782645e+01,  1.47474217e+01,  9.42334441e+00, ...,
        -2.24387301e-02,  1.66561261e-01, -2.88825439e-01],
       [ 1.55219270e+01,  1.38831670e+01,  1.37819839e-01, ...,
         6.42988622e-02, -1.93326843e-01,  2.91391603e-02],
       ...,
       [-1.09626258e+01, -8.17530173e+00,  1.57517580e+00, ...,
        -1.26789338e-01, -4.51287775e-02, -1.48021688e-01],
       [-1.45418380e+01,  1.48301918e+01,  1.81446462e+00, ...,
         1.44989075e-01, -3.43279757e-01,  1.44158737e-02],
       [-1.09278178e+01,  1.25420771e+01,  1.46593310e+00, ...,
         1.89395219e-02, -2.54982265e-01, -6.31800067e-02]])

In [13]:
import joblib

# Save each component
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(pca, 'pca.pkl')
joblib.dump(model, 'svm_model.pkl')


['svm_model.pkl']

In [14]:
#note to future self, this is how ur supposed to transform ur input as well

scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")
model = joblib.load("svm_model.pkl")

# Apply same preprocessing
X_scaled = scaler.transform(X_test)
X_pca = pca.transform(X_scaled)

# Predict
y_pred = model.predict(X_pca)
print("Predictions:", y_pred)

C:\Users\saiet\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 100 features, but StandardScaler is expecting 271 features as input.